In [1]:
import matplotlib.image as mp_image
import matplotlib.pyplot as plt
import subprocess
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from functools import partial
sess = tf.Session()

In [2]:
def extract_filename(full_name, filename_extension):
    return full_name.split(sep="."+filename_extension)[0].split(sep='/')[-1]

extract_png_filename = partial(extract_filename, filename_extension='png')

## 讀取訓練資料到陣列

In [3]:
# ground truth
df_UNBC_pain_score = pd.read_csv('./UNBC_pain_score_table.csv') # 讀取疼痛分數資料
df_UNBC_pain_score.sort_values(by=['PSPI'], ascending=False, inplace=True) # 高分排前面
df_UNBC_pain_score.reset_index(drop=True, inplace=True) # 更新 index

In [4]:
# feed training data

# 建構檔案路徑
# croped_face_folder = '../datalake/crop_face_from_UNBC_pain/'
file_ext = '.png'
# 讀取目錄底下所有檔案絕對路徑
paths_croped_face_img = subprocess.getoutput('find /home/lky/crop_face_from_UNBC_pain_GCP/ -type f -name "*.png"').split('\n')
print("count = {}".format(+len(paths_croped_face_img)))
# paths_croped_face_img

count = 46376


In [5]:
# 只使用有被偵測到臉的
df_croped_face_img = pd.DataFrame(
                        data = {'full_name':paths_croped_face_img, 
                                'file_name':[i for i in map(extract_png_filename, paths_croped_face_img)]})

In [6]:
# 合併檔案路徑表和分數表，之後只用這張表，清空不必要變數
df_croped_face_img = df_croped_face_img.merge(right=df_UNBC_pain_score, on='file_name', how='inner')
paths_croped_face_img = []
df_UNBC_pain_score = []
print(df_croped_face_img.index.size)

46376


In [7]:
# 流程測試用
# df_croped_face_img = df_croped_face_img[:1000]
print(df_croped_face_img.index.size)

46376


In [8]:
# tensorflow 讀取影像
import skimage.measure
training_dataset = []
read_fail_set = []
print("read img {:5d}, {}".format(len(training_dataset), subprocess.getoutput('date')))

#-------------------------------------------------------------------

for img_full_name in df_croped_face_img['full_name']:
    try:
        raw_img = cv2.imread(img_full_name)
    except:
        read_fail_set.append(img_full_name)
        print(img_full_name, len(read_fail_set))

    gray_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY) #轉灰
    gray_img = cv2.equalizeHist(gray_img) #亮度正規化
    norm_size_img = cv2.resize(gray_img,(64, 64)) #統一尺寸
    norm_size_img = skimage.measure.block_reduce(norm_size_img, (2,2), np.max) #最大池化
    reshape_img = np.reshape(norm_size_img, [-1]) #拉平
    training_dataset.append(reshape_img) #push in list
    
    if(0 == len(training_dataset)%5000):
        print("read img {:5d}, {}".format(len(training_dataset), subprocess.getoutput('date')))

read img     0, 四  9月  7 14:03:03 CST 2017
read img  5000, 四  9月  7 14:03:18 CST 2017
read img 10000, 四  9月  7 14:03:33 CST 2017
read img 15000, 四  9月  7 14:03:47 CST 2017
read img 20000, 四  9月  7 14:04:01 CST 2017
read img 25000, 四  9月  7 14:04:16 CST 2017
read img 30000, 四  9月  7 14:04:29 CST 2017
read img 35000, 四  9月  7 14:04:43 CST 2017
read img 40000, 四  9月  7 14:04:57 CST 2017
read img 45000, 四  9月  7 14:05:11 CST 2017


## Neural Network construct

In [9]:
# 定義輸出入變數
input_layer = tf.placeholder(dtype=tf.float32, shape=[None, 1024], name='input_layer')
ground_truth = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='ground_truth')

In [65]:
# 建構神經網路

# 單層建構函式
def make_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

in_size = 1024
out_size = 1 

# 定義輸出入變數
xs = tf.placeholder(tf.float32, [None, in_size])
ys = tf.placeholder(tf.float32, [None, out_size])

# 定義多層流程
my_activation_function = tf.nn.tanh # 統一中間層活化函數
layer_hidden_1 = make_layer(xs, in_size, 32, activation_function=my_activation_function)
layer_output   = make_layer(layer_hidden_1, 32, out_size) 

In [83]:
# 定義損失函數
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - layer_output)))

In [84]:
# 定義學習方法
train_step = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

In [85]:
# 變數初始化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [86]:
# 訓練
for epoch in range(5):
    train_squence = np.linspace(
                                0,len(training_dataset),len(training_dataset),
                                dtype=int, endpoint=False)
    np.random.shuffle(train_squence)
    for i in train_squence:
        _ = sess.run([train_step], feed_dict = {
                                            xs: [training_dataset[i]],
                                            ys: [[df_croped_face_img['PSPI'][i]]]})
            
    cost = sess.run([loss], feed_dict = {
                                    xs: [training_dataset[i]],
                                    ys: [[df_croped_face_img['PSPI'][i]]]})
    print("epoch = {}, cost = {}, {}".format(epoch, cost, subprocess.getoutput('date')))

epoch = 0, cost = [0.0029676273], 四  9月  7 15:14:20 CST 2017
epoch = 1, cost = [0.28248096], 四  9月  7 15:14:44 CST 2017
epoch = 2, cost = [0.005077899], 四  9月  7 15:15:09 CST 2017
epoch = 3, cost = [2.3907268], 四  9月  7 15:15:33 CST 2017
epoch = 4, cost = [0.23021726], 四  9月  7 15:15:58 CST 2017


In [90]:
! ls

crop face to file from UNBC pain dataset.ipynb	 train_net_by_pain_face.ipynb
reconstruct UNBC pain database PSPI index.ipynb  UNBC_pain_score_table.csv
trained_model


In [93]:
saver = tf.train.Saver()
saver_path = saver.save(sess, "trained_model/train_net_by_pain_face.ckpt")
print(saver_path)

trained_model/train_net_by_pain_face.ckpt
